## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sci

from sklearn.metrics import confusion_matrix

## Exercise 9.11.)

Do a classification analysis on the rootstock data of Table 6.2 as follows:

1. Find the linear classification functions.
2. Find the classification table using the linear classification functions in part 1 (assuming $\Sigma_1 = \Sigma_2 = \Sigma_3$).
3. Find the classification table using quadratic classification functions (assuming population covariance matrices are not equal).
4. Find the classification table using the nearest neighbor method.
5. Find the classification table using a kernel density estimator 

### Setup

In [2]:
names = ["type"] + [f"y{i+1}" for i in range(4)]
df = pd.read_table("Data/T6_2_ROOT.DAT", header=None, sep="\s+", names=names)

In [3]:
df.head(10)

,type,y1,y2,y3,y4
0,1,1.11,2.569,3.58,0.760
1,1,1.19,2.928,3.75,0.821
2,1,1.09,2.865,3.93,0.928
3,1,1.25,3.844,3.94,1.009
4,1,1.11,3.027,3.60,0.766
5,1,1.08,2.336,3.51,0.726
6,1,1.11,3.211,3.98,1.209
7,1,1.16,3.037,3.62,0.750
8,2,1.05,2.074,4.09,1.036
9,2,1.17,2.885,4.06,1.094


In [4]:
type_l = list(set(df["type"]))
type_l

[1, 2, 3, 4, 5, 6]

In [5]:
df_typed = [df[df["type"] == i][["y1", "y2", "y3", "y4"]] for i in type_l]

In our case for all classes we have the same number of trees:

In [6]:
for df_i in df_typed:
    print(df_i.shape[0])

8
8
8
8
8
8


In [7]:
n = df_typed[0].shape[0]

In [8]:
S = []
for df_i in df_typed:
    S.append(df_i.cov().to_numpy())
S = np.array(S)

In [9]:
S_pl = n/( df.shape[0] - len(type_l) ) * np.sum(S, axis=0)

In [10]:
S_pl

array([[0.00870714, 0.046165  , 0.01507721, 0.00590857],
       [0.046165  , 0.33041606, 0.11873776, 0.05742078],
       [0.01507721, 0.11873776, 0.1167568 , 0.06752806],
       [0.00590857, 0.05742078, 0.06752806, 0.04687142]])

In [20]:
y_mean = []
for df_i in df_typed:
    y_mean.append(df_i.mean().to_numpy())

y_mean = np.array(y_mean)

### 1-2

In [21]:
def L(y):
    ans = []
    for i in range(len(type_l)):
        ans.append(y_mean[i].T @ np.linalg.inv(S_pl) @ y - 0.5 * y_mean[i].T @ np.linalg.inv(S_pl) @ y_mean[i])
    return np.array(ans)

In [22]:
def linear_classify(y):
    return type_l[np.argmax(L(y))]

In [23]:
df["lin_class"] = df[["y1", "y2", "y3", "y4"]].apply(linear_classify, axis=1)

In [24]:
answer = confusion_matrix(df["type"], df["lin_class"])
print(answer)

[[5 0 0 1 0 2]
 [0 3 2 1 2 0]
 [0 0 6 1 1 0]
 [3 0 1 4 0 0]
 [0 3 1 0 3 1]
 [2 0 0 0 2 4]]


In [25]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.5208333333333334

### 3

In [31]:
def Q(y):
    ans = []
    for i in range(len(type_l)):
        ans.append(-0.5 * np.log(np.linalg.det(S[i])) - 0.5 * (y - y_mean[i]).T @ np.linalg.inv(S[i]) @ (y - y_mean[i]))
    return np.array(ans)

In [32]:
def quadratic_classify(y):
    return type_l[np.argmax(Q(y))]

In [33]:
df["quad_class"] = df[["y1", "y2", "y3", "y4"]].apply(quadratic_classify, axis=1)

In [34]:
answer = confusion_matrix(df["type"], df["quad_class"])
print(answer)

[[8 0 0 0 0 0]
 [0 7 0 1 0 0]
 [1 0 6 0 1 0]
 [0 0 1 7 0 0]
 [0 3 0 0 4 1]
 [2 0 0 0 1 5]]


In [35]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.7708333333333334

### 4

In [36]:
def dist(y1, y2):
    return (y1 - y2).T @ np.linalg.inv(S_pl) @ (y1 - y2)

In [74]:
def knn(y, k=3):
    dist_list = []
    for i in range(df.shape[0]):
        yi = df[["y1", "y2", "y3", "y4"]].to_numpy()[i]
        dist_list.append(dist(y, yi))
        
    nearest = np.argsort(np.array(dist_list))
    
    return nearest[:k]

In [128]:
def knn_classify(y, k=3):
    # equal n_i are assumed, as in our situation
    # in case of equal k_i we choose the nearest
    nearest = knn(y, k)
    type_nearest = df["type"][nearest]
    type_nearest = list(type_nearest)
    
    return max(set(type_nearest), key=type_nearest.count)

In [129]:
def knn_classify_with_ties(y, k=3):
    # collect information about ties in the process
    global ties_l
    
    nearest = knn(y, k)
    type_nearest = df["type"][nearest]
    type_nearest = list(type_nearest)
    
    apperances = np.array([type_nearest.count(type_l[i]) for i in range(len(type_l))])
    
    winner = np.argmax(apperances)
    if apperances[winner] == np.sort(apperances)[-2]:
        ties_l[type_nearest[0]-1] += 1
        
    return type_l[winner]

In [124]:
df["knn_class"] = df[["y1", "y2", "y3", "y4"]].apply(knn_classify, axis=1)

In [125]:
answer = confusion_matrix(df["type"], df["knn_class"])
print(answer)

[[6 0 0 2 0 0]
 [0 7 0 0 1 0]
 [1 0 6 0 1 0]
 [3 0 0 5 0 0]
 [0 1 0 0 6 1]
 [2 0 0 1 0 5]]


In [127]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.7291666666666666

In [126]:
ties_l = np.zeros(len(type_l))
df["knn_class_ties"] = df[["y1", "y2", "y3", "y4"]].apply(knn_classify_with_ties, axis=1)
ties_l

array([1., 3., 0., 3., 0., 1.])

### 5

In [145]:
def f_hat(y, h=1, p=4):
    # normal kernel
    
    coef = 1/(df.shape[0] * h**p * (np.linalg.det(S_pl))**0.5)
    
    ans = np.zeros(len(type_l))
    for i in range(len(type_l)):
        for yj in df_typed[i].to_numpy():
            ans[i] += np.exp(-(y - yj).T @ np.linalg.inv(S_pl) @ (y - yj) / (2 * h ** 2))
    
    return coef * ans

def kde_classify(y):
    return type_l[np.argmax(f_hat(y))]

In [146]:
df["kde_class"] = df[["y1", "y2", "y3", "y4"]].apply(kde_classify, axis=1)

In [147]:
answer = confusion_matrix(df["type"], df["kde_class"])
print(answer)

[[8 0 0 0 0 0]
 [0 7 1 0 0 0]
 [1 0 6 0 1 0]
 [1 0 0 7 0 0]
 [0 1 0 0 6 1]
 [3 0 0 0 0 5]]


In [148]:
correct_rate = np.sum(np.diag(answer))/(np.sum(answer))
correct_rate

0.8125